Census of Drug and Alcohol Treatment Services in Northern Ireland:Breakdown by Service Type

In [1]:
from gssutils import *
if is_interactive():
    import requests
    from cachecontrol import CacheControl
    from cachecontrol.caches.file_cache import FileCache
    from cachecontrol.heuristics import LastModified
    from pathlib import Path

    session = CacheControl(requests.Session(),
                           cache=FileCache('.cache'),
                           heuristic=LastModified())

    sourceFolder = Path('in')
    sourceFolder.mkdir(exist_ok=True)

    inputURL = 'https://www.health-ni.gov.uk/sites/default/files/publications/dhssps/data-census-drug-alcohol-treatment-services.xlsx'
    inputFile = sourceFolder / 'data-census-drug-alcohol-treatment-services.xlsx'
    response = session.get(inputURL)
    with open(inputFile, 'wb') as f:
      f.write(response.content)
    tab = loadxlstabs(inputFile, sheetids='Table 3')[0]

Loading in/data-census-drug-alcohol-treatment-services.xlsx which has size 46265 bytes
Table names: ['Table 3']


In [2]:
observations = tab.excel_ref('B16').expand(DOWN).expand(RIGHT).is_not_blank() - tab.excel_ref('B22').expand(DOWN).expand(RIGHT)  


In [3]:
observations

{<F19 '*'>, <M21 3.5>, <F18 '*'>, <I20 88.6>, <I16 '*'>, <C20 99.1>, <M17 5341.0>, <D20 91.8>, <E20 96.1>, <K18 45.0>, <L16 225.0>, <E19 '-'>, <L20 79.6>, <K21 2.2>, <B20 98.9>, <L21 3.8>, <E16 '-'>, <H20 76.3>, <L19 16.6>, <G16 '*'>, <I19 '*'>, <H17 810.0>, <B18 '*'>, <C17 321.0>, <K16 27.0>, <I18 '*'>, <F20 88.8>, <M18 211.0>, <G21 '*'>, <F17 2203.0>, <H16 '*'>, <B21 '*'>, <L18 51.0>, <I21 '*'>, <J18 115.0>, <J20 89.1>, <J16 165.0>, <D17 270.0>, <K20 96.5>, <G18 '*'>, <M19 7.0>, <B17 94.0>, <G19 '*'>, <M16 417.0>, <H21 '*'>, <J19 6.4>, <I17 4656.0>, <K17 1964.0>, <H18 '*'>, <E17 685.0>, <F16 '*'>, <J21 4.5>, <H19 '*'>, <L17 1080.0>, <E21 '*'>, <K19 1.3>, <G20 96.0>, <B19 '-'>, <M20 89.5>, <G17 1643.0>, <E18 '*'>, <J17 2297.0>, <B16 '-'>, <F21 '*'>}

In [4]:
Service = tab.excel_ref('A15').expand(DOWN).is_not_blank()
Service

{<A20 'Non-residential (%)'>, <A16 'Residential'>, <A21 'Mixed (%)'>, <A19 'Residential (%)'>, <A15 'Total'>, <A18 'Mixed'>, <A17 'Non-residential '>}

In [5]:
Treatment = tab.excel_ref('B14').expand(RIGHT)
Treatment

{<H14 'Drugs & Alcohol'>, <K14 'Drugs Only'>, <B14 'Alcohol Only'>, <C14 'Drugs Only'>, <I14 'Total'>, <L14 'Drugs & Alcohol'>, <M14 ''>, <J14 'Alcohol Only'>, <F14 'Alcohol Only'>, <D14 'Drugs & Alcohol'>, <G14 'Drugs Only'>, <E14 'Total'>}

In [6]:
age = tab.excel_ref('B13').expand(RIGHT).is_not_blank()
age

{<F13 '18 and over'>, <B13 'Under 18 '>, <J13 'Treatment Type'>, <M13 'Overall Total'>}

In [7]:
Dimensions = [
            HDim(Treatment,'Treatment Type',DIRECTLY,ABOVE),
            HDim(Service,'Residential Status',DIRECTLY,LEFT),
            HDim(age,'Age',CLOSEST,LEFT),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','People'),
            HDimConst('Sex','Persons'),
            HDimConst('Period','1 March 2017')
            ]

In [8]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
# savepreviewhtml(c1)

In [9]:
new_table = c1.topandas()
new_table.loc[new_table['Age'] == 'Treatment Type', 'Age'] = 'All Ages'
new_table.loc[new_table['Age'] == 'Overall Total', 'Age'] = 'All Ages'
new_table.loc[new_table['Treatment Type'] == '', 'Treatment Type'] = 'Total'
new_table

,OBS,DATAMARKER,Treatment Type,Residential Status,Age,Measure Type,Unit,Sex,Period
0,,-,Alcohol Only,Residential,Under 18,Count,People,Persons,1 March 2017
1,,-,Total,Residential,Under 18,Count,People,Persons,1 March 2017
2,,*,Alcohol Only,Residential,18 and over,Count,People,Persons,1 March 2017
3,,*,Drugs Only,Residential,18 and over,Count,People,Persons,1 March 2017
4,,*,Drugs & Alcohol,Residential,18 and over,Count,People,Persons,1 March 2017
5,,*,Total,Residential,18 and over,Count,People,Persons,1 March 2017
6,165,NaN,Alcohol Only,Residential,All Ages,Count,People,Persons,1 March 2017
7,27,NaN,Drugs Only,Residential,All Ages,Count,People,Persons,1 March 2017
8,225,NaN,Drugs & Alcohol,Residential,All Ages,Count,People,Persons,1 March 2017
9,417,NaN,Total,Residential,All Ages,Count,People,Persons,1 March 2017


In [10]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [11]:
new_table.dtypes

Value                 object
DATAMARKER            object
Treatment Type        object
Residential Status    object
Age                   object
Measure Type          object
Unit                  object
Sex                   object
Period                object
dtype: object

In [12]:
new_table.tail(5)

,Value,DATAMARKER,Treatment Type,Residential Status,Age,Measure Type,Unit,Sex,Period
59,,*,Total,Mixed (%),18 and over,Count,People,Persons,1 March 2017
60,4.5,NaN,Alcohol Only,Mixed (%),All Ages,Count,People,Persons,1 March 2017
61,2.2,NaN,Drugs Only,Mixed (%),All Ages,Count,People,Persons,1 March 2017
62,3.8,NaN,Drugs & Alcohol,Mixed (%),All Ages,Count,People,Persons,1 March 2017
63,3.5,NaN,Total,Mixed (%),All Ages,Count,People,Persons,1 March 2017


In [13]:
new_table.count()

Value                 64
DATAMARKER            24
Treatment Type        64
Residential Status    64
Age                   64
Measure Type          64
Unit                  64
Sex                   64
Period                64
dtype: int64

In [14]:
new_table = new_table[new_table['Value'] !=  0 ]

In [15]:
new_table = new_table[new_table['Value'] !=  '' ]

In [16]:
new_table.count()

Value                 40
DATAMARKER             0
Treatment Type        40
Residential Status    40
Age                   40
Measure Type          40
Unit                  40
Sex                   40
Period                40
dtype: int64

In [17]:
def user_perc(x):
    
    if str(x) == 'Treatment Type':
        return 'All years'
    else:
        return x
    
new_table['Age'] = new_table.apply(lambda row: user_perc(row['Age']), axis = 1)

In [18]:
new_table['Treatment Type'].fillna('Total', inplace = True)

In [19]:
new_table['Service Type'] = 'All'
# new_table['Residential Status'] = 'All'
new_table['Health and Social Care Trust']  = 'All'

In [20]:
new_table = new_table[['Period', 'Sex', 'Age', 'Service Type', 'Residential Status', 'Treatment Type', 'Health and Social Care Trust', 'Measure Type', 'Unit', 'Value']]

In [21]:
new_table.head(5)

,Period,Sex,Age,Service Type,Residential Status,Treatment Type,Health and Social Care Trust,Measure Type,Unit,Value
6,1 March 2017,Persons,All Ages,All,Residential,Alcohol Only,All,Count,People,165
7,1 March 2017,Persons,All Ages,All,Residential,Drugs Only,All,Count,People,27
8,1 March 2017,Persons,All Ages,All,Residential,Drugs & Alcohol,All,Count,People,225
9,1 March 2017,Persons,All Ages,All,Residential,Total,All,Count,People,417
10,1 March 2017,Persons,Under 18,All,Non-residential,Alcohol Only,All,Count,People,94
